# Exploratory

In this notebook we explore the Spanish split of the [MIRACL corpus](https://arxiv.org/abs/2210.09984).

In [30]:
from datasets import load_dataset

dataset = load_dataset("miracl/miracl-corpus", "es")

dataset

DatasetDict({
    train: Dataset({
        features: ['docid', 'title', 'text'],
        num_rows: 10373953
    })
})

In [33]:
dataset = dataset.rename_column("text", "contents")

In [4]:
dataset["train"][1]

{'docid': '9894682#0',
 'title': 'Flapjack (postre)',
 'text': 'Un flapjack es un dulce de origen inglés, parecido a una barrita energética. Se hace con copos de avena, mantequilla, azúcar de caña y de un tipo de melaza llamado sirope dorado, un ingrediente de la cocina británica.'}

Copio de la descripción del corpus

```text
The docid has the schema X#Y, where all passages with the same X come from the same Wikipedia article, whereas Y denotes the passage within that article,numbered sequentially. The text field contains the text of the passage. The title field contains the name of the article the passage comes from.
```

En este caso, el artículo es el 9894682 y el pasaje es el 0. El título es "Flapjack".

Vamos a bajar las queries ahora

In [6]:
miracl = load_dataset('miracl/miracl', "es", use_auth_token=True)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 564.41it/s]
Generating dev split: 648 examples [10:03,  1.07 examples/s]
Generating testB split: 1515 examples [10:06,  2.50 examples/s]
Generating train split: 2162 examples [10:11,  3.53 examples/s]


In [7]:
miracl

DatasetDict({
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 648
    })
    testB: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 1515
    })
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 2162
    })
})

In [8]:

miracl["train"][1]

{'query_id': '919762#0',
 'query': '¿Qué significa Cuauhtémoc?',
 'positive_passages': [{'docid': '17725#1',
   'text': 'El nombre Cuauhtémoc que significa literalmente \'Águila que descendió (se posó)\' (náhuatl: "kwāw"("-tli") \'águila\', "temō" \'descender\', "-k" ). La forma honorífica de Cuauhtémoc es Cuauhtemoctzin (el sufijo "-tzin" se usa para designar una dignidad similar a "Don" o "Señor" en español).',
   'title': 'Cuauhtémoc'},
  {'docid': '544740#1',
   'text': 'Toma su nombre de la Avenida Cuauhtémoc y esta a su vez de Cuauhtémoc, último tlatoani de México-Tenochtitlan. El isotipo de la estación, diseñado por el taller de Lanz Wyman, es un águila ya que Cuauhtémoc significa "águila que cae".Existen conexiones con las estaciones y paradas de diversos sistemas de transporte:',
   'title': 'Cuauhtémoc (estación)'},
  {'docid': '7967208#1',
   'text': 'La localidad toma el nombre de "Cuauhtémoc", que en náhuatl significa ‘el águila se posó’.',
   'title': 'Cuauhtémoc (Actopan

Este es el que tenemos que usar para referencia

In [9]:
for k in range(10):
    ex = miracl["train"][k]
    negatives = ex["negative_passages"]
    positives = ex["positive_passages"]

    print(len(negatives), len(positives))

7 3
5 5
4 6
8 2
4 6
7 3
8 2
3 7
8 2
6 4


## BM25 index

Creamos índice BM25 siguiendo [esta guía](https://github.com/castorini/pyserini/blob/master/docs/usage-index.md)

In [35]:


dataset["train"].to_json("../data/miracl.json", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/10374 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 10374/10374 [01:08<00:00, 151.05ba/s]


5104086234

In [41]:

from pyserini.search import LuceneSearcher

LuceneSearcher("../data/miracl")

JavaException: JVM exception occurred: ../data/miracl does not exist or is not a directory. java.lang.IllegalArgumentException

Error!